In [ ]:
!ls

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]
# plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 24
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["font.family"] = "serif"

In [ ]:
FONT_SIZE_TITLE_PLOT = 48  # 40
FONT_SIZE_TITLE_AX = 36  # 30
FONT_SIZE_LABEL = 30  # 24
FONT_SIZE_TICKS = 24  # 20
FONT_SIZE_LEGEND = 32  # 28

In [ ]:
PROJECT_FOLDER = "PycharmProjects/thesis-gan"

In [ ]:
stock_names = ["PEP", "KO", "NVDA", "KSU"]
stock_names = ["KO", "PEP"]
n_stocks = len(stock_names)

In [ ]:
RUN_ID_PRICE = "24uxrxqz"

In [ ]:
SIGMA_SCALERs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

In [ ]:
PATH_PICKLE_PRICE_REALS = f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/reals.pickle"
PATH_PICKLE_PRICE_NOPER = (
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/perturbation_sigmascaler=None.pickle"
)

with open(PATH_PICKLE_PRICE_REALS, "rb") as handle:
    dict_reals = pickle.load(handle)
with open(PATH_PICKLE_PRICE_NOPER, "rb") as handle:
    dict_no_per = pickle.load(handle)

prices_no_per = dict_no_per["pred_prices"].numpy()
prices_reals = dict_reals["prices"][:, : prices_no_per.shape[-1]]

prices_reals.shape, prices_no_per.shape

In [ ]:
PATHs_PICKLE_PRICE_PER = [
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/perturbation_sigmascaler={sigma_scaler}.pickle"
    for sigma_scaler in SIGMA_SCALERs
]

l = list()
for path in PATHs_PICKLE_PRICE_PER:
    with open(path, "rb") as handle:
        dict_per = pickle.load(handle)
    prices_per = dict_per["pred_prices"].numpy()
    l.append(prices_per)

prices_per = np.asarray(l).transpose(1, 0, 2)
prices_per.shape

In [ ]:
history_indexes = np.arange(390)
continuation_indexes = np.arange(390, prices_reals.shape[-1])
history_indexes.shape, continuation_indexes.shape

In [ ]:
iteration_perturbation = 9
start_perturbation_n1 = 390 + 150 * iteration_perturbation
start_perturbation_n2 = start_perturbation_n1 + 150

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 9))
axes = axes.ravel()

add_label = True
for (
    ax,
    stock_name,
    price_real,
    price_no_per,
    price_per,
) in zip(axes, stock_names, prices_reals, prices_no_per, prices_per):
    ax.plot(history_indexes, price_no_per[:390], color="C0", label="Observed" if add_label else None)
    ax.plot(continuation_indexes, price_real[390:], color="C1", label="Real" if add_label else None)

    ax.plot(continuation_indexes, price_no_per[390:], color="C3", label="W/o perturbation" if add_label else None)

    for i, (price_per_s, sigma_scaler) in enumerate(zip(price_per, SIGMA_SCALERs)):
        ax.plot(
            continuation_indexes,
            price_per_s[390:],
            color=f"C{i+4}",
            label=f"W/ perturbation: {sigma_scaler}" if add_label else None,
        )

    ax.axvline(x=390, color="r")

    min_ = min(price_real.min(), price_no_per.min(), price_per.min())
    max_ = max(price_real.max(), price_no_per.max(), price_per.max())
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation_n1, min_),
            width=start_perturbation_n2 - start_perturbation_n1,
            height=max_ - min_,
            color="r",
            alpha=0.3,
        )
    )
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation_n2, min_),
            width=start_perturbation_n2 - start_perturbation_n1,
            height=max_ - min_,
            color="g",
            alpha=0.3,
        )
    )

    ax.set_title(stock_name, fontsize=FONT_SIZE_TITLE_AX)
    ax.set_xlabel("Steps", fontsize=FONT_SIZE_LABEL)
    ax.set_ylabel("Price ($)", fontsize=FONT_SIZE_LABEL, rotation=90)
    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=FONT_SIZE_TICKS)
    ax.set_xticks([0, 390, start_perturbation_n1, start_perturbation_n2, start_perturbation_n2 + 150])

    add_label = False

fig.suptitle("Prices", fontsize=FONT_SIZE_TITLE_PLOT, y=1)
fig.legend(loc="upper center", ncol=3, fontsize=FONT_SIZE_LEGEND, frameon=False, bbox_to_anchor=(0.5, 1.3))
fig.tight_layout()
# plt.savefig(f"{PROJECT_FOLDER}/plot_finali/multistock/perturbations/perturbed_KO.pdf")
plt.show()
plt.close(fig)

In [ ]:
corr_real = np.corrcoef(prices_reals)[0, 1]
corr_real = round(corr_real, 2)
corr_real

In [ ]:
KOwoPer, PEPwoPer = prices_no_per[:, start_perturbation_n2 + 150 :]
corr_KOwoPer_PEPwoPer = np.corrcoef(KOwoPer, PEPwoPer)[0, 1]
corr_KOwoPer_PEPwoPer = round(corr_KOwoPer_PEPwoPer, 2)
corr_KOwoPer_PEPwoPer

In [ ]:
KOwPer, PEPwPer = prices_per[:, :, start_perturbation_n2 + 150 :]
corrs_KOwPerS_PEPwoPer = list()
for KOwPerS, sigma in zip(KOwPer, SIGMA_SCALERs):
    corr_KOwPerS_PEPwoPer = np.corrcoef(KOwPerS, PEPwoPer)[0, 1]
    corr_KOwPerS_PEPwoPer = round(corr_KOwPerS_PEPwoPer, 2)
    corrs_KOwPerS_PEPwoPer.append(corr_KOwPerS_PEPwoPer)
corrs_KOwPerS_PEPwoPer

In [ ]:
corrs_KOwPerS_PEPwPerS = list()
for KOwPerS, PEPwPerS, sigma in zip(KOwPer, PEPwPer, SIGMA_SCALERs):
    corr_KOwPerS_PEPwPerS = np.corrcoef(KOwPerS, PEPwPerS)[0, 1]
    corr_KOwPerS_PEPwPerS = round(corr_KOwPerS_PEPwPerS, 2)
    corrs_KOwPerS_PEPwPerS.append(corr_KOwPerS_PEPwPerS)
corrs_KOwPerS_PEPwPerS

In [ ]:
print("corr_real:\t\t\t\t", corr_real)
print("corr_KOwoPer_PEPwoPer:\t", corr_KOwoPer_PEPwoPer)
print()
print("sigma\tcorrs_KOwPerS_PEPwoPer\tcorrs_KOwPerS_PEPwPerS")
for sigma, corr_KOwPerS_PEPwoPer, corr_KOwPerS_PEPwPerS in zip(
    SIGMA_SCALERs, corrs_KOwPerS_PEPwoPer, corrs_KOwPerS_PEPwPerS
):
    print(sigma, "\t\t", corr_KOwPerS_PEPwoPer, "\t\t\t\t\t\t", corr_KOwPerS_PEPwPerS)